# Clusterização de Textos com a Lógica do Algoritmo de Butina

Este notebook faz uma tentativa de adaptar o algoritmo de clusterização de Darko Butina, originalmente projetado para química computacional, para agrupar documentos de texto, especificamente, sinopses de livros.

### O Algoritmo Original de Butina

O algoritmo foi projetado para ser uma maneira rápida e automatizada de agrupar grandes conjuntos de dados moleculares. Suas etapas principais são:

1.  **Geração de "Fingerprints"**: Cada molécula é convertida em uma representação binária (uma string de 1s e 0s) que descreve suas características estruturais.
2.  **Identificação de Centroides Potenciais**:
    * O algoritmo calcula, para cada molécula, o número de "vizinhos" que ela possui acima de um limiar de similaridade (usando o índice de Tanimoto).
    * As moléculas são ordenadas em ordem decrescente pelo número de vizinhos. As que estão no topo da lista são as melhores candidatas a centroides.
3.  **Clusterização por Esferas de Exclusão**:
    * O algoritmo seleciona a primeira molécula não clusterizada da lista ordenada como um novo centroide.
    * Todos os vizinhos similares a este centroide são agrupados com ele.
    * Uma vez que uma molécula é atribuída a um cluster, ela é "marcada" e não pode fazer parte de nenhum outro (a "esfera de exclusão").
    * O processo se repete até que todas as moléculas sejam avaliadas. Moléculas não agrupadas tornam-se "singletons".
  
Referência: https://ptabdata.blob.core.windows.net/files/2016/PGR2016-00042/v58_1061%20-%20Butina.pdf

In [2]:
synopses_grande = [
    # --- Gênero: Fantasia / Aventura ---
    "Um jovem mago descobre seu destino ao frequentar uma escola de magia e bruxaria.",
    "Uma sociedade secreta de bruxos luta contra as forças das trevas para proteger o mundo.",
    "Uma antiga profecia envia um herói improvável em uma jornada para encontrar uma espada mítica.",
    "Elfos e anões devem deixar de lado suas diferenças para reconquistar um reino perdido de um dragão.",
    "Uma garota descobre que pode falar com criaturas míticas e deve protegê-las de um mundo que se industrializa.",
    "Um poderoso necromante ameaça libertar um exército de mortos-vivos sobre os reinos.",

    # --- Gênero: Ficção Científica / Distopia ---
    "Num futuro distópico, uma garota se voluntaria para um jogo mortal televisionado para salvar sua irmã.",
    "Após o colapso da sociedade, jovens são forçados a competir em uma arena brutal.",
    "A última colônia humana viaja pelo espaço em uma nave-geração, enfrentando um motim e uma ameaça externa desconhecida.",
    "Em um mundo onde as emoções são suprimidas por lei, um homem começa a sentir e precisa fugir antes de ser 'curado'.",
    "Um clone descobre a verdade sobre sua criação e lidera uma rebelião contra seus mestres.",
    "Megacorporações governam o mundo a partir de torres de neon, e um hacker descobre uma conspiração que pode derrubá-las.",

    # --- Gênero: Romance ---
    "Dois amantes de classes sociais diferentes enfrentam obstáculos em sua busca pelo amor.",
    "Um homem rico e orgulhoso se apaixona por uma mulher de espírito livre, apesar de suas diferenças.",
    "Um romance de verão se transforma em um amor para a vida toda.",
    "Dois chefs rivais se apaixonam enquanto competem em um concurso de culinária de alto risco.",
    "Uma historiadora viaja no tempo e se apaixona por um cavaleiro, sabendo que seu amor é impossível através dos séculos.",
    "A dona de uma livraria encontra bilhetes de amor anônimos e inicia uma busca por seu admirador secreto.",
    "Um casamento arranjado entre nobres de reinos rivais inesperadamente se transforma em amor verdadeiro.",

    # --- Gênero: Mistério / Suspense ---
    "Um detetive brilhante, mas excêntrico, resolve crimes complexos em Londres.",
    "Quando um assassinato ocorre em um trem luxuoso, um detetive famoso deve encontrar o culpado entre os passageiros.",
    "A morte misteriosa de um bilionário da tecnologia em seu quarto do pânico trancado intriga um inspetor veterano.",
    "Um ladrão de arte deixa pistas para um detetive aposentado, atraindo-o para um último jogo de gato e rato.",
    "Uma podcaster que reabre um caso arquivado começa a receber ameaças, percebendo que o assassino está entre seus ouvintes.",
    "O xerife de uma cidade pequena investiga uma série de desaparecimentos bizarros que os locais atribuem a uma lenda folclórica.",

    # --- Gênero: Ficção Histórica ---
    "Um gladiador na Roma antiga luta por sua liberdade, conspirando contra o imperador que traiu sua família.",
    "Durante o Renascimento, um jovem artista se envolve na intriga política da família Médici em Florença.",
    "Uma espiã da resistência opera na Paris ocupada durante a Segunda Guerra Mundial, arriscando tudo por seu país.",

    # --- Gênero: Terror ---
    "Uma família se muda para uma casa nova apenas para descobrir que ela é assombrada pelos espíritos de seus antigos moradores.",
    "Uma expedição científica na Antártida descobre uma antiga criatura metamorfa que os caça um por um."
]

### Conjuntos de Palavras e Similaridade Jaccard

O **Índice de Tanimoto** para dados binários é matematicamente equivalente à **Similaridade Jaccard**.

* **Fingerprint**: Um conjunto (`set`) contendo as palavras únicas de cada sinopse.
* **Similaridade**: A Similaridade Jaccard, que mede a sobreposição de vocabulário (interseção / união) entre dois conjuntos de palavras.

> Obs: O TFIDF está sendo utilizado apenas como tokenizador

In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

def jaccard_similarity(set1, set2):
    """Calcula a similaridade Jaccard entre dois conjuntos."""
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def cluster_text_jaccard(docs, similarity_threshold):
    """
    Clusteriza textos usando a lógica de Butina com Similaridade Jaccard.
    """
    # Etapa 1: "Fingerprinting" com conjuntos de palavras
    
    stopwords_pt = stopwords.words('portuguese')
    tokenizer = TfidfVectorizer(stop_words=stopwords_pt).build_analyzer()
    doc_sets = [set(tokenizer(doc)) for doc in docs]

    # Etapa 2: Calcular a matriz de similaridade Jaccard
    num_docs = len(docs)
    sim_matrix = np.zeros((num_docs, num_docs))
    for i in range(num_docs):
        for j in range(num_docs):
            sim_matrix[i, j] = jaccard_similarity(doc_sets[i], doc_sets[j])

    # Etapa 3: Aplicar a lógica de Butina, pega os que têm mais vizinhos e ordena
    num_neighbors = (sim_matrix >= similarity_threshold).sum(axis=1) - 1
    sorted_indices = np.argsort(num_neighbors)[::-1]

    # Etapa 4: Agrupar os documentos em clusters ou singletons, excluindo os já agrupados (flagged)
    clusters = {}
    flagged = np.zeros(len(docs), dtype=bool)
    cluster_id = 0

    for idx in sorted_indices:
        if flagged[idx]:
            continue

        cluster_id += 1
        clusters[cluster_id] = {'centroid_synopsis': docs[idx], 'members': []}
        flagged[idx] = True

        neighbor_indices = np.where((sim_matrix[idx] >= similarity_threshold) & (np.arange(num_docs) != idx))[0]
        for neighbor_idx in neighbor_indices:
            if not flagged[neighbor_idx]:
                clusters[cluster_id]['members'].append(docs[neighbor_idx])
                flagged[neighbor_idx] = True

    singleton_indices = np.where(flagged == False)[0]
    singletons = [docs[i] for i in singleton_indices]

    return clusters, singletons

print("--- RESULTADOS COM SIMILARIDADE JACCARD ---")
clusters_jac, singletons_jac = cluster_text_jaccard(synopses_grande, similarity_threshold=0.05)

for cid, data in clusters_jac.items():
    print(f"\n📚 Cluster {cid}")
    print(f"   Centroide: \"{data['centroid_synopsis']}\"")
    print("   Membros:")
    if data['members']:
        for member in data['members']:
            print(f"    - \"{member}\"")
    else:
        print("     (Nenhum outro membro neste cluster)")

print(f"\n--- Singletons ---")
for s in singletons_jac:
    print(f"  - \"{s}\"")

--- RESULTADOS COM SIMILARIDADE JACCARD ---

📚 Cluster 1
   Centroide: "Uma garota descobre que pode falar com criaturas míticas e deve protegê-las de um mundo que se industrializa."
   Membros:
    - "Um jovem mago descobre seu destino ao frequentar uma escola de magia e bruxaria."
    - "Uma sociedade secreta de bruxos luta contra as forças das trevas para proteger o mundo."
    - "Num futuro distópico, uma garota se voluntaria para um jogo mortal televisionado para salvar sua irmã."
    - "Um clone descobre a verdade sobre sua criação e lidera uma rebelião contra seus mestres."
    - "Megacorporações governam o mundo a partir de torres de neon, e um hacker descobre uma conspiração que pode derrubá-las."
    - "Quando um assassinato ocorre em um trem luxuoso, um detetive famoso deve encontrar o culpado entre os passageiros."
    - "Uma expedição científica na Antártida descobre uma antiga criatura metamorfa que os caça um por um."

📚 Cluster 2
   Centroide: "Um casamento arranjado en

In [5]:

def recomendar_por_sinopse(sinopse_usuario, clusters, threshold=0.1):
    """
    Dada uma sinopse, encontra o cluster mais similar e recomenda outros livros desse cluster.
    """
    stopwords_pt = stopwords.words('portuguese')
    tokenizer = TfidfVectorizer(stop_words=stopwords_pt).build_analyzer()
    set_usuario = set(tokenizer(sinopse_usuario))
    
    melhor_cluster = None
    melhor_sim = 0
    
    for cid, data in clusters.items():
        set_centroide = set(tokenizer(data['centroid_synopsis']))
        sim = jaccard_similarity(set_usuario, set_centroide)
        if sim > melhor_sim and sim >= threshold:
            melhor_sim = sim
            melhor_cluster = cid

    if melhor_cluster is not None:
        recomendacoes = [data['centroid_synopsis']] + clusters[melhor_cluster]['members']
        # Remove a própria sinopse do usuário, se estiver na lista
        recomendacoes = [r for r in recomendacoes if r != sinopse_usuario]
        return recomendacoes
    else:
        return []

sinopse_usuario = "Um romance com muitos desafios e um amor que supera barreiras sociais."
recomendacoes = recomendar_por_sinopse(sinopse_usuario, clusters_jac, threshold=0.05)
print("\n--- Recomendações para o usuário ---")
for r in recomendacoes:
    print(f"  - \"{r}\"")


--- Recomendações para o usuário ---
  - "Uma espiã da resistência opera na Paris ocupada durante a Segunda Guerra Mundial, arriscando tudo por seu país."
  - "Um poderoso necromante ameaça libertar um exército de mortos-vivos sobre os reinos."
  - "Dois amantes de classes sociais diferentes enfrentam obstáculos em sua busca pelo amor."
  - "Um romance de verão se transforma em um amor para a vida toda."
  - "Dois chefs rivais se apaixonam enquanto competem em um concurso de culinária de alto risco."
  - "Uma historiadora viaja no tempo e se apaixona por um cavaleiro, sabendo que seu amor é impossível através dos séculos."
  - "A dona de uma livraria encontra bilhetes de amor anônimos e inicia uma busca por seu admirador secreto."
